# Aggregate anomaly labels from top-5 solutions from the ASHRAE competition

In [9]:
import pandas as pd

#### 1. Process the outlier lables from first ranked solution 
- The file sol1_bad_meter_readings.csv contains the outliers (is_bad_meter_reading == 1) 

In [10]:
rank1 = pd.read_csv('data/rank1_bad_meter_readings.csv')
print(rank1.info())
rank1.columns = ['rank1_outlier']
rank1.rank1_outlier.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 1 columns):
is_bad_meter_reading    int64
dtypes: int64(1)
memory usage: 154.2 MB
None


0    18901626
1     1314474
Name: rank1_outlier, dtype: int64

#### 2. Process the outlier lables from the second ranked solution 
- The file sol2_rows_to_drop.csv contains the index to the outlier rows

In [11]:
rank2_drop = pd.read_csv('data/rank2_rows_to_drop.csv', header=None, names = ['rank2_outlier'])
print(rank2_drop.info())

rank2 = rank1.copy()
rank2.columns = ['rank2_outlier']
rank2.rank2_outlier = 0
rank2.iloc[rank2_drop.rank2_outlier] = 1
rank2.rank2_outlier.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986260 entries, 0 to 986259
Data columns (total 1 columns):
rank2_outlier    986260 non-null int64
dtypes: int64(1)
memory usage: 7.5 MB
None


0    19229840
1      986260
Name: rank2_outlier, dtype: int64

#### 3. Process the outlier lables from third ranked solution 

In [16]:
# train1 = pd.read_feather('./data/rank3_train_cleanup_001.feather')
# print(train1.info())
rank3_drop = pd.read_csv('./data/rank3_rows_to_drop.csv', header=None, names = ['rank3_outlier'])
print(rank3_drop.info())

rank3 = rank1.copy()
rank3.columns = ['rank3_outlier']
rank3.rank3_outlier = 0
rank3.iloc[rank3_drop.rank3_outlier] = 1
rank3.rank3_outlier.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525023 entries, 0 to 525022
Data columns (total 1 columns):
rank3_outlier    525023 non-null int64
dtypes: int64(1)
memory usage: 4.0 MB
None


0    19691077
1      525023
Name: rank3_outlier, dtype: int64

#### 4. Process the outlier lables from fourth ranked solution 

In [39]:
rank4_exp = pd.read_pickle('./data/rank4_train_exception.pkl')
rank4_exp.to_csv('./data/rank4_train_exception.csv')
print(rank4_exp.info())
rank4_exp.exception.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 1 columns):
exception    int64
dtypes: int64(1)
memory usage: 154.2 MB
None


 0    16258788
-1     1923669
-3     1051924
 3      452505
 1      365029
-2       85465
 2       78720
Name: exception, dtype: int64

In [40]:
rank4_train = pd.read_pickle('./data/rank4_train_df.pkl')
print(rank4_train.info())
rank4_train.exception.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 33 columns):
building_id                    int16
meter                          int8
timestamp                      datetime64[ns]
meter_reading                  float64
DT_M                           int8
DT_W                           int8
DT_D                           int16
DT_hour                        int8
DT_day_week                    int8
DT_day_month                   int8
DT_week_month                  int8
site_id                        int8
primary_use                    int8
square_feet                    int32
year_built                     float16
floor_count                    float16
air_temperature                float64
cloud_coverage                 float16
dew_temperature                float64
precip_depth_1_hr              float16
sea_level_pressure             float64
wind_direction                 float16
wind_speed                     float64
offset        

 0    16255437
-1     1923669
-3     1051924
 3      452505
 1      365029
-2       85465
 2       78720
 4        3351
Name: exception, dtype: int64

In [47]:
# https://github.com/buds-lab/ashrae-great-energy-predictor-3-solution-analysis/blob/master/solutions/rank-4/model1/fork-of-as-2kfold-model6-xgb-fr7d12-fold0.ipynb
tr_x = rank4_train.copy()
tr_x = tr_x[(((tr_x.site_id == 0) & (tr_x.exception != 1) & (tr_x.exception != 3)) | (tr_x.site_id != 0)) & (tr_x.exception != 4)]
print(rank4_train.shape, tr_x.shape, rank4_train.shape[0]-tr_x.shape[0])
tr_x.exception.value_counts()

(20216100, 33) (19857254, 33) 358846


 0    16255437
-1     1923669
-3     1051924
 1      359291
 3      102748
-2       85465
 2       78720
Name: exception, dtype: int64

In [48]:
#https://github.com/buds-lab/ashrae-great-energy-predictor-3-solution-analysis/blob/master/solutions/rank-4/model2/as-meter2-no-1099-xgb-meter0-fold0.ipynb
tr_x = rank4_train.copy()    
tr_x = tr_x[(tr_x.exception != 3) & (tr_x.exception != 1) & (tr_x.exception != 4)]
print(rank4_train.shape, tr_x.shape, rank4_train.shape[0]-tr_x.shape[0])

tr_x.exception.value_counts()

(20216100, 33) (19395215, 33) 820885


 0    16255437
-1     1923669
-3     1051924
-2       85465
 2       78720
Name: exception, dtype: int64

In [50]:
# https://github.com/buds-lab/ashrae-great-energy-predictor-3-solution-analysis/blob/master/solutions/rank-4/model3/ashrae-kfold-lightgbm-without-leak-1-08.ipynb
train = pd.read_csv('./data/train.csv')

train_df = train.copy()
train_df = train_df.copy()[train_df['building_id'] != 1099]
train_df = train_df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

print(train.shape, train_df.shape, train.shape[0] - train_df.shape[0])

(20216100, 4) (19852422, 4) 363678


In [62]:
rank4 = rank1.copy()
rank4.columns = ['rank4_outlier']
rank4.rank4_outlier = 1
rank4.iloc[tr_x.index] = 0
rank4.rank4_outlier.value_counts()

0    19395215
1      820885
Name: rank4_outlier, dtype: int64

#### 5. Process the outlier lables from fifth ranked solution 

In [53]:
##TODO..

#### Merge all anomaly labels

In [63]:
df_anomaly = pd.concat([rank1, rank2, rank3, rank4])
print(df_anomaly.info())
df_anomaly.head()

C:\Users\sbbpan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 80864400 entries, 0 to 20216099
Data columns (total 4 columns):
rank1_outlier    float64
rank2_outlier    float64
rank3_outlier    float64
rank4_outlier    float64
dtypes: float64(4)
memory usage: 3.0 GB
None


,rank1_outlier,rank2_outlier,rank3_outlier,rank4_outlier
0,1.0,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN
3,1.0,NaN,NaN,NaN
4,1.0,NaN,NaN,NaN


In [64]:
df_anomaly.describe().T

,count,mean,std,min,25%,50%,75%,max
rank1_outlier,20216100.0,0.065021,0.246563,0.0,0.0,0.0,0.0,1.0
rank2_outlier,20216100.0,0.048786,0.215420,0.0,0.0,0.0,0.0,1.0
rank3_outlier,20216100.0,0.025971,0.159047,0.0,0.0,0.0,0.0,1.0
rank4_outlier,20216100.0,0.040606,0.197375,0.0,0.0,0.0,0.0,1.0


In [65]:
df_anomaly.to_csv('./data/anomaly_labels.csv', index=False)